In [5]:
!pip install pyspark

     -------------------------------------- 281.4/281.4 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 199.7/199.7 kB 6.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824023 sha256=bd06028ef32e46a104cb7e87e836a4058f7dc40b5c00ba00c0c809033daf6431
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\6c\e3\9b\0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [8]:
from pyspark.sql import SparkSession
spark=(SparkSession
.builder
.appName("sunny")
.getOrCreate())

In [9]:
import pyspark.sql.functions as f

In [10]:
sc=spark.sparkContext
sqlContext=spark

In [13]:
!git clone https://github.com/jonesberg/DataAnalysisWithPythonAndPySpark-Data.git


'git' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
book=spark.read.text("/content/DataAnalysisWithPythonAndPySpark-Data/gutenberg_books/1342-0.txt")

AnalysisException: Path does not exist: file:/content/DataAnalysisWithPythonAndPySpark-Data/gutenberg_books/1342-0.txt

In [14]:
import os

In [15]:
DIRECTORY="/content/DataAnalysisWithPythonAndPySpark-Data/broadcast_logs"
logs=spark.read.csv(
os.path.join(DIRECTORY,"/content/DataAnalysisWithPythonAndPySpark-Data/broadcast_logs/BroadcastLogs_2018_Q3_M8_sample.CSV"),
sep="|",
header=True,
inferSchema=True,
timestampFormat="yyyy-MM-dd",
)

AnalysisException: Path does not exist: file:/content/DataAnalysisWithPythonAndPySpark-Data/broadcast_logs/BroadcastLogs_2018_Q3_M8_sample.CSV

In [ ]:
logs=spark.read.csv(
os.path.join(DIRECTORY,"/content/DataAnalysisWithPythonAndPySpark-Data/broadcast_logs/BroadcastLogs_2018_Q3_M8_sample.CSV"),
sep="|",
header=True,
inferSchema=True,
timestampFormat="yyyy-MM-dd",
)

In [ ]:
logs

In [ ]:
logs.toPandas

In [ ]:
#tree format show
logs.printSchema()

In [ ]:
logs.select("BroadcastLogID","LogServiceID","Language2","DubDramaCreditID","LogDate").show(5,False)

In [ ]:
logs.select("BroadcastLogID","LogServiceID","LogDate")
logs.select(*["BroadcastLogID","LogServiceID","LogDate"])
logs.select(
     f.col("BroadCastLogID"),f.col("LogServiceID"),f.col("LogDate")
)
logs.select(
    *[f.col("BroadCastLogID"),f.col("LogServiceID"),f.col("LogDate")]
)


In [ ]:
import numpy as np

In [ ]:
column_split=np.array_split(np.array(logs.columns),len(logs.columns)// 3)
print(column_split)

In [ ]:
for i in logs.columns:
  logs.select(i).summary().show()

In [ ]:
for x in column_split:
  logs.select(*x).show(5,False)


In [ ]:
Data_src=DIRECTORY
wholeRDD=sc.wholeTextFiles(DIRECTORY)
wholeRDD.take(1)

In [ ]:
myRDD=sc.parallelize([1,2,3,4,5,6])
print("this RDD has", myRDD.count(),"element")
print("the first element is", myRDD.first())
print("the first 2 element are", myRDD.take(2))
print("the entire collection is",myRDD.collect())

In [ ]:
myRDD=sc.parallelize([1,2,3,4,5,6])
myRDD.saveAsTextFile("myFirstRDD")

In [ ]:
myRDD=sc.parallelize([1,2,3,4,5,6])
myRDDsum=myRDD.reduce(lambda l, r: l+r)
print("The sum of my RDD is", myRDDsum)

In [ ]:
def wow_print(e):
  print("-"*(len(e)+4))
  print("*",e,"*")
  print("-"*(len(e)+4))
my_fruits=["apples","oranges","pear",
"banana"]
dataRDD = sc.parallelize(my_fruits)
print("The output of foreach is:", 
dataRDD.foreach(wow_print))

In [ ]:
print(my_fruits)

In [ ]:
logs.select(
    f.col("Duration"),
    (
        f.col("Duration").substr(1,2).cast("int")*60 *60
     +f.col("Duration").substr(4,2).cast("int")*60 
     +f.col("Duration").substr(7,2,).cast("int")
    ).alias("Duration_second"),
).distinct().show(5)

In [ ]:
logs=logs.withColumn(
    "Duration_seconds",
    (
      f.col("Duration").substr(1,2).cast("int")*60 *60
     +f.col("Duration").substr(4,2).cast("int")*60 
     +f.col("Duration").substr(7,2,).cast("int")
    ),
)
logs.printSchema()

In [ ]:
logs.toDF(*[x.lower() for x in logs.columns]).printSchema()

In [ ]:
logs.select(sorted(logs.columns)).printSchema()

In [ ]:
for i in logs.columns:
  logs.describe(i).show()

In [ ]:
flatRDD=dataRDD\
  .flatMap(lambda line: line.split(' ')).union(dataRDD)
for line in flatRDD.collect():
  print(line)

In [ ]:
my_fruits=["apple oranges pear", "banana apple"]
dataRDD=sc.parallelize(my_fruits)
print("number of element is the RDD:", dataRDD.count())
print("it contains the following string:", dataRDD.collect())

In [ ]:
fruits1=["apple","orange","pera"]
fruits2=["banana","orange","apple"]
fruits1RDD=sc.parallelize(fruits1)
fruits2RDD=sc.parallelize(fruits2)

fruits1RDD.union(fruits2RDD).collect()

In [ ]:
fruits1RDD.subtract(fruits2RDD).collect()

In [ ]:
fruits1RDD.cartesian(fruits2RDD).collect()

In [ ]:
fruits1RDD.subtract(fruits2RDD).collect()

In [ ]:
myRDD=sc.parallelize(range(100))
print(myRDD.sample(False,0.3).collect())

In [ ]:
myRDD=sc.parallelize(range(100))
print(myRDD.sample(True,0.3,123).collect())

In [ ]:
data_src=["having","fun","learning","spark"]
script_path="/home/student/Data/repeat.sh"
myRDD=sc.parallelize(data_src)
pipeRDD=myRDD.pipe(script_path)


In [ ]:
my_fruits=["apples","oranges","pear","banana"]
pairRDD=sc.parallelize(my_fruits)\
          .keyBy(lambda fruit: len(fruit))
for i in pairRDD.collect():
  print(i)

In [ ]:
fav_fruits=[("henry","apples grapes banana"),
            ("shaun","watermelon strawberry"),
            ("sharon","pear apples kiwi",)
]
pairRDD=sc.parallelize(fav_fruits)\
          .flatMapValues(lambda fruits: fruits.split(" "))
for i in pairRDD.collect():
  print(i)           